<a href="https://colab.research.google.com/github/elaheJ/AnnaKarinena_BookChat_GeminiContext/blob/main/AnnaKarenina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%%capture
!pip install -q BeautifulSoup
!pip install torch
!!pip install -q gradio

In [5]:
import torch
import os
import google.generativeai as genai
from google.generativeai import caching
import datetime
import time
from google.colab import userdata
myGKey = userdata.get("GOOGLE_API_KEY")
genai.configure(api_key=myGKey)

In [6]:
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config
)

In [7]:
#By Paul Mooney https://www.kaggle.com/code/paultimothymooney/how-to-upload-large-files-to-gemini-1-5/notebook

def upload_to_gemini(path, mime_type=None):
  """Uploads the given file to Gemini.

  See https://ai.google.dev/gemini-api/docs/prompting_with_media
  """
  file = genai.upload_file(path, mime_type=mime_type)
  print(f"Uploaded file '{file.display_name}' as: {file.uri}")
  return file

def wait_for_files_active(files):
  """Waits for the given files to be active.

  Some files uploaded to the Gemini API need to be processed before they can be
  used as prompt inputs. The status can be seen by querying the file's "state"
  field.

  This implementation uses a simple blocking polling loop. Production code
  should probably employ a more sophisticated approach.
  """
  print("Waiting for file processing...")
  for name in (file.name for file in files):
    file = genai.get_file(name)
    while file.state.name == "PROCESSING":
      print(".", end="", flush=True)
      time.sleep(10)
      file = genai.get_file(name)
    if file.state.name != "ACTIVE":
      raise Exception(f"File {file.name} failed to process")

In [9]:
#By Paul Mooney https://www.kaggle.com/code/paultimothymooney/how-to-upload-large-files-to-gemini-1-5/notebook
text='/content/drive/MyDrive/Gutenberg/Books/Tolstoy/AnnaKarenina.txt'
files = [
  upload_to_gemini(text, mime_type="text/plain"),
]

wait_for_files_active(files)

chat_session = model.start_chat(
  history=[
    {
      "role": "user",
      "parts": [
        files[0],
      ],
    }
  ]
)

Uploaded file 'AnnaKarenina.txt' as: https://generativelanguage.googleapis.com/v1beta/files/ynr0edxrgmgo
Waiting for file processing...


In [10]:
import gradio as gr
def geminiChatter(message, history):
    response = chat_session.send_message(message)
    return response.text
with gr.Blocks(theme=gr.themes.Soft) as app:
      gr.ChatInterface(fn=geminiChatter, type="messages")
app.launch(debug=True, share=True)

/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:1027: UserWarning: Theme should be a class loaded from gradio.themes
  warnings.warn("Theme should be a class loaded from gradio.themes")


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://025d8ccae32d1c0ae2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://025d8ccae32d1c0ae2.gradio.live
